In [1]:
! pip install bitsandbytes -q
! pip install torchao -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.1 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
import torch
from transformers import TorchAoConfig, Gemma3ForConditionalGeneration, AutoProcessor
from torch.utils.data import DataLoader, TensorDataset, Dataset
import json

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
sheet_path = 'Ka-ChatBot_BenchMark.xlsx'
faq_df = pd.read_excel(sheet_path, sheet_name="faq")
samples_df = pd.read_excel(sheet_path, sheet_name="samples")

faqs = faq_df['faq'].tolist()
ids = faq_df['idx'].tolist()
categories = faq_df['category'].tolist()

samples = samples_df['sample'].tolist()
labels = torch.tensor(samples_df['gt_idx'].tolist())

faq_list = zip(ids, faqs, categories)


In [4]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model_name = "google/gemma-3-4b-it"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="cuda",
    quantization_config=quantization_config,
    # attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
# tokenizer.padding_side = 'left'
# tokenizer.pad_token = tokenizer.eos_token
# model.generation_config.pad_token_id = tokenizer.pad_token_id

In [46]:
instruction = """You are given a user question in Persian, a list of predefined questions with IDs, and the category each question belongs to. Your task is to find the single most semantically similar question.

Steps:
1. Determine which category the user question belongs to. Focus on meaning and intent.
2. Compare the user question only to questions within that category.
3. Choose the question whose intent or meaning best matches the user question.
"""
category_group = {}
for (id, q, cat) in zip(ids, faqs, categories):
  category_group[cat] = category_group.get(cat, []) + [(id, q)]
faq_list = "Predefined questions:\n"
for cat in category_group.keys():
  faq_list += f"\n\nCategory: {cat}\n" + '\n'.join(f"id: {id} - {q}" for id, q in category_group[cat])

rules = """Rules:
- All questions are in Persian. Do not translate, only compare.
- Return only a JSON object in the format: {"id": "<id>"}
- Do not include explanations or any other text."""


sysprompt = f"{instruction}\n\n{faq_list}\n\n{rules}"

In [14]:
# best performace
instruction = """You are given a user’s question and a list of predefined questions with IDs.
Your task is to find the most semantically similar predefined question."""

faq_list = "Predefined questions:\n" + '\n'.join(f"id: {id} - {q}" for (id, q, cat) in zip(ids, faqs, categories))

rules = """Rules:
- Compare based on meaning, not just keywords.
- All questions are in Persian. Do not translate, only compare.
- Return only a JSON object in the format: {"id": "<id>"}.
- Do not include explanations or any other text."""


sysprompt = f"{instruction}\n\n{faq_list}\n\n{rules}"

In [ ]:
instruction = """You are given a user question in Persian, a list of predefined questions with IDs, and the category each question belongs to. Your task is to find the single most semantically similar question."""
category_group = {}
for (id, q, cat) in zip(ids, faqs, categories):
  category_group[cat] = category_group.get(cat, []) + [(id, q)]
faq_list = "Predefined questions:\n"
for cat in category_group.keys():
  faq_list += f"\n\nCategory: {cat}\n" + '\n'.join(f"id: {id} - {q}" for id, q in category_group[cat])

rules = """

Steps:
0. All questions are in Persian. Do not translate, only compare.
1. Determine which category the user question belongs to. Focus on meaning and intent.
2. Compare the user question only to questions within that category.
3. Choose the question whose intent or meaning best matches the user question.
4. Return only a JSON object in the format: {"id": "<id>"}
5. Do not include explanations or any other text."""


sysprompt = f"{instruction}\n\n{faq_list}\n\n{rules}"

In [47]:
print(sysprompt)

You are given a user question in Persian, a list of predefined questions with IDs, and the category each question belongs to. Your task is to find the single most semantically similar question.

Steps:
1. Determine which category the user question belongs to. Focus on meaning and intent.
2. Compare the user question only to questions within that category.
3. Choose the question whose intent or meaning best matches the user question.


Predefined questions:


Category: احراز هویت
id: 1 - چرا من در احراز هویت افتتاح حساب خطای عدم تطبیق تصویر می گیرم؟ 

Category: افتتاح حساب
id: 2 - جگونه افتتاح حساب غیر حضوری کنم ؟
id: 3 - چگونه میتوانم حساب خودم را ببندم؟

Category: پرداخت قبض با حساب
id: 4 - چگونه پرداخت قبض با حساب/کارت انجام دهم؟

Category: تغییر تلفن همراه
id: 5 - چرا در زمان کارت به کارت اعلام می کند شماره در سامانه شاهکار ثبت نشده

Category: مسدودی کارت
id: 6 - چگونه کارت را از مسدودی در بیاورم؟
id: 7 - چگونه کارت خود را مسدود کنم؟ 

Category: تسهیلات در جریان و پرداخت قسط
id: 8 -

In [48]:
class QuestionDataset(Dataset):
  def __init__(self, questions, system_prompt):
    self.questions = questions
    self.system_prompt = system_prompt

  def __len__(self):
    return len(self.questions)

  def __getitem__(self, idx):
    messages = [
      {"role": "system", "content": self.system_prompt},
      {"role": "user", "content": self.questions[idx]},
    ]
    return messages


In [49]:
def collate_fn(batch):
  return tokenizer.apply_chat_template(
      batch, padding=True,
      add_generation_prompt=True,
      return_tensors='pt',
      return_dict=True).to(device)

In [50]:
userqs_ds = QuestionDataset(samples, sysprompt)
dataloader = DataLoader(userqs_ds, shuffle=False, batch_size=16, collate_fn=collate_fn)

In [51]:
from tqdm import tqdm

In [52]:
allresponses = []
with torch.no_grad():
  for inputs in tqdm(dataloader):
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    new_tokens = outputs[:, inputs["input_ids"].shape[-1]:-1]
    responses = tokenizer.batch_decode(new_tokens, skip_special_tokens=True)
    allresponses = allresponses + responses


100%|██████████| 6/6 [02:38<00:00, 26.46s/it]


In [54]:
pids = []
for resp in allresponses:
  try:
    resp = resp.strip('`').strip('json').strip('\n')
    pids.append(int(json.loads(resp)['id']))
  except:
    pids.append(0)
pids = torch.tensor(pids)

In [55]:
pids.count_nonzero()

tensor(93)

In [57]:
(pids == labels).sum() / len(labels)

tensor(0.6237)

In [59]:
resultdf = pd.read_csv('results.csv')
resultdf['google/gemma-3-4b-it'] = pids
resultdf.to_csv('newresults.csv', index=False)